## This is the Final version implemented with reinforcement learning neural network (RLN).

In [1]:
import pygame as pg
import numpy as np
#import imp
from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
Nx = 8
grid_size = 64
Lx = 600
Ly = 600
side = 1
max_eva = 100
red_win = False
white_win = False
alpha = -200.
beta = -alpha
global king_value
king_value = 2
global man_value
man_value = 1

margin_x = (Lx-Nx*grid_size)//2
margin_y = (Ly-Nx*grid_size)//2

step_fr_w = 9 #forward_right
step_fl_w = 7 #forward_left
step_br_w = -7 #backward_rigt
step_bl_w = -9 #backward_left

step_fr_r = -7 #forward_right
step_fl_r = -9 #forward_left
step_br_r = 9 #backward_right
step_bl_r = 7 #backward_left

print text [Reference](https://home.gamer.com.tw/creationDetail.php?sn=3669589)

[Reference 1](https://ithelp.ithome.com.tw/articles/10209416)
[Reference 2](https://gist.github.com/kingknight0921/594251488548e1cdae7c)
[Reference 3](https://ithelp.ithome.com.tw/articles/10209416)

### Initialize the board

In [3]:
iE_w = np.arange(0,12)
iE_r = np.arange(20,32)

x_w = (2*iE_w)%Nx
y_w = ((2*iE_w)//Nx)
x_r = (2*iE_r)%Nx
y_r = ((2*iE_r)//Nx)

parity = (x_w+y_w+1)%2
x_w += parity
parity = (x_r+y_r+1)%2
x_r += parity

man_w = x_w+ Nx*y_w
man_w = list(man_w)
man_r = x_r+ Nx*y_r
man_r = list(man_r)

king_w = []
king_r = []

In [4]:
import Basic_Operation
import Pygame_Plotting
import Mini_Max_and_Alpha_Beta
import DQN_DIY

#imp.reload(DQN_DIY) # for reloading if DQN_DIY.py has any modification

Using TensorFlow backend.


### Computer use mini-max and alpha-beta prouning

In [13]:
pg.init()

FLIP = False
FPS = 30 # frame per second
#clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append(0)
    
#man_r = [19]
#man_w = [49,44]
#king_r = [28]
#king_w = [24]    
#man_r = [10,42,44,46]
#man_w = [12,1,30]
#king_r = [26,28]
#king_w = [39]
BOARD = Pygame_Plotting.Plotting(Nx, grid_size, margin_x, margin_y, Lx, Ly, man_value, king_value, FLIP, WINDOWS)
CHECKERS = Basic_Operation.Checkers_Basic(Nx, man_value, king_value, step_fr_r, step_fl_r, step_br_r, step_bl_r, \
                                          step_fr_w, step_fl_w, step_br_w, step_bl_w, man_r, man_w, king_r, king_w)
COMPUTER_MOVE = Mini_Max_and_Alpha_Beta.prouning(Nx, max_eva, man_value, king_value, CHECKERS)

checkers = CHECKERS.CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#################
BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
pg.display.flip()

click= []
save = []

side = -1
total_move = 1
if side == 1:
    red_first = True
else:
    red_first = False

depth = 12
point = 0.0
running = True
continue_to_jump = False

if side==1:
    print('Red Turn')
    CHECKERS.saving(checkers, save)
else:
    print('White Trun')
print('==============================================')

while running:
    event = pg.event.wait()
    mods = pg.key.get_mods()
    if event.type == MOUSEBUTTONDOWN and event.button==1:
        (x,y) = pg.mouse.get_pos()
        column = (x-margin_x)//grid_size
        row = (y-margin_y)//grid_size
        index = column + row*Nx
        if FLIP:
            index = Nx**2-1-index
        click.append(index)
    # take back
    elif not continue_to_jump and event.type == KEYDOWN and mods&pg.KMOD_CTRL:  # See https://www.pygame.org/docs/ref/key.html
        if event.key == pg.K_z:
            if len(save)>1:
                checkers = save[-2].copy()  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                            # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html
                del save[-1]  # remove the last element
        
                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip() 
                print('\tRed takes back.\n==============================================\nRed Turn')
                total_move -= 1
        elif event.key == pg.K_f:
            FLIP = not FLIP
            print('Board flipped.')
            BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
            BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
            pg.display.flip() 
     
    # close the window
    elif event.type == pg.QUIT:
        running = False
        
    if not continue_to_jump:
        (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = CHECKERS.AVAILABLE_MOVE(side, checkers)
    available_move = man_jump+king_jump+man_walk+king_walk
    # draw out the hints
    (available_start,available_stop) = BOARD.MOVE_HINT(available_move)  # find the avaiable start points and stop points, then drawing it out as hint
    if side==1:
        for i in available_start:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,0,255),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        for i in available_stop:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,255,0),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        pg.display.flip()
    # have a winner
    if available_move == []:
        if side==1:
            white_win = True
            print('No move.')
            print('White win!')
        else:
            red_win = True
            print('No move.')
            print('Red win!')
        break

    # Computer moves
    if side==-1:
        print('\tComputer is thinking...')
        if point == max_eva and depth>2:
            depth -= 2
        #checkers, point = COMPUTER_MOVE.FIND_BEST_MOVE_MIN_MAX(-1, checkers, depth)  #mini-max
        checkers, point = COMPUTER_MOVE.FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta) #alpha-beta
        print('\tEvaluation = %.2f ; Depth = %d .' %(point,depth) )
        print('\tComputer finishes thinking.')
        if red_first:
            print(f'Move {total_move} Ends')
            total_move += 1
        print('Red Turn\n==============================================')
        
        side = -side
        BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
        BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP) 
        pg.display.flip()
        CHECKERS.saving(checkers,save)

    # read the last two clicked square and see whether it corresponding any available move
    elif side==1 and len(click)>1:
        if man_jump+king_jump!=[]:
            # man jump
            if tuple(click[-2:]) in man_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = 0
                checkers[taken] = 0
                checkers[stop] = -man_value
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                CHECKERS.MAN_JUMP_RED(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # red man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == 0:
                        checkers[stop] = -king_value #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    side = -side
                    if not red_first:
                        print(f'Move {total_move} Ends')
                        total_move += 1
                    print('White Trun\n==============================================')
                    click = [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece) 
                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king jmup
            elif tuple(click[-2:]) in king_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = 0
                checkers[taken] = 0
                checkers[stop] = -king_value
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                CHECKERS.KING_JUMP_RED(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    side = -side
                    if not red_first:
                        print(f'Move {total_move} Ends')
                        total_move += 1
                    print('White Trun\n==============================================')
                    click= [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece)

                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
        #man walk    
        elif man_walk+king_walk!=[]:
            if tuple(click[-2:]) in man_walk:
                start = click[-2]
                stop = click[-1]  
                checkers[start] = 0
                if stop//Nx == 0:
                    checkers[stop] = -king_value #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = -man_value
                    print('\tMan walks from %d to %d.' %(start,stop))
                side = -side
                if not red_first:
                    print(f'Move {total_move} Ends')
                    total_move += 1
                print('White Trun\n==============================================')
                click= []
                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king walk
            elif tuple(click[-2:]) in king_walk:
                start = click[-2]
                stop = click[-1]              
                checkers[start] = 0
                checkers[stop] = -king_value
                print('\tKing walks from %d to %d.' %(start,stop))
                side = -side
                if not red_first:
                    print(f'Move {total_move} Ends')
                    total_move += 1
                print('White Trun\n==============================================')
                click= []
                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip()
pg.quit()

White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man walks from 42 to 33.
Move 1 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man jumps from 49 to 35.
Move 2 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man walks from 35 to 26.
Move 3 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man jumps from 44 to 26.
Move 4 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man jumps from 40 to 26.
Move 5 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Red takes back.
Red Turn
	Red takes back.
Red Turn
	Red takes back.
Red Turn
	Red takes back.
Red Turn
	Red takes back.
Red Turn
	Man walks from 42 to 33.
Move 1 Ends


	Man walks from 10 to 3 .
	Man promotes
Move 28 Ends
White Trun
	Computer is thinking...
	Evaluation = -1.05 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	King walks from 3 to 12.
Move 29 Ends
White Trun
	Computer is thinking...
	Evaluation = -1.05 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	King walks from 19 to 26.
Move 30 Ends
White Trun
	Computer is thinking...
	Evaluation = 0.00 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	King walks from 12 to 19.
Move 31 Ends
White Trun
	Computer is thinking...
	Evaluation = 1.06 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man walks from 44 to 37.
Move 32 Ends
White Trun
	Computer is thinking...
	Evaluation = 1.06 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	King walks from 19 to 28.
Move 33 Ends
White Trun
	Computer is thinking...
	Evaluation = 6.50 ; Depth = 4 .
	Computer finishes thinking.
Red Turn
	Man jumps from 62 to 44.
Move 34 Ends
White Trun
	Computer is thinking...
	Evaluation = 100.00 ; Depth =

### End game technique trained by QLN

為了簡化訓練，我們設定白方持兩顆棋，紅方持一顆，所有的棋都已經升變成國王。開始時，紅方的棋子會被產生在第 55 格，白方的棋子會在棋盤上的剩下 30 格被隨機產生( 31 格減掉1格一開始就會被吃掉的位置)，訓練的目標是要讓白方突破平局，得勝的要領則是要讓白方其中一隻國王佔領第 55 或第 62 格，才有機會把紅色的國王逼出來，所以在訓練的時候走到這兩格會給正的 reward 並結束一個 epsoide，同時靠近紅方的棋子、或是之後有必勝的走法(用 $\alpha$-$\beta$ prouning 評估)也會給正的 reward，但如果重複走同樣的棋步或下一步會被對方吃掉，則會導致 epsoide 結束並得到負的 reward，並根據被吃掉的棋子是一顆或兩顆判斷是平手或輸棋。在這樣的簡化模型下，我們只要讓神經網路學習移動就好，不會有吃子甚至連吃的問題，所以 action space 大小是有限且固定的。最後把總共的贏棋、平手、輸棋的概率列出來。


DQN 的神經網路採用輸入層 64(對應棋盤上的64格，有白色王的格子 =2 ，紅色王的格子 =-2，其餘則 =0 )，中間兩層 256、128 的隱藏層，最後連接到 8 個神經元的輸出層，對應到 8 個可能的動作(每顆白色王可以右上、右下、左上、左下移動)， optimizer 用 adam ， loss function 採 mse, 所有隱藏層都用 relu 當 activation function。如果在訓練時神經網路預測出來的動作不在 available move 裡面，則重新預測一次；在測試的時候，則是看輸出的動作 array (有 8 個元素)哪個值最大且對應到符合規則的移動，就選擇該動作。

In [21]:
RL = DQN_DIY.DeepQNetwork(8, 64,
                  Nx,
                  king_value,
                  step_fr_r,
                  step_fl_r,
                  step_br_r,
                  step_bl_r,
                  step_fr_w,
                  step_fl_w,
                  step_br_w,
                  step_bl_w,
                  learning_rate=0.1,
                  reward_decay=0.9,
                  e_greedy=0.9,
                  replace_target_iter=200,
                  memory_size=200
                  )

step = 0
total_win = 0
total_lose = 0
total_draw = 0
total_eposide = 1000

for episode in range(total_eposide):
###################################    
    checkers = []
    for i in range(0,Nx*Nx):
        checkers.append(0)
    
    man_r = []
    man_w = []
    king_r, king_w = RL.POSITION_INITIALIZE()
###################################

    CHECKERS = Basic_Operation.Checkers_Basic(Nx, man_value, king_value, step_fr_r, step_fl_r, step_br_r, step_bl_r, \
                                          step_fr_w, step_fl_w, step_br_w, step_bl_w, man_r, man_w, king_r, king_w)
    COMPUTER_MOVE = Mini_Max_and_Alpha_Beta.prouning(Nx, max_eva, man_value, king_value, CHECKERS)

    checkers = CHECKERS.CHECKER_LIST(checkers) # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).  
    #RL.model1.load_weights('./Checkers_RL_Model/RL_model1_1st_weights.h5')
    #RL.model2.load_weights('./Checkers_RL_Model/RL_model2_1st_weights.h5')
    RL.model1.load_weights('./Checkers_RL_Model/RL_model1_2nd_weights.h5')
    RL.model2.load_weights('./Checkers_RL_Model/RL_model2_2nd_weights.h5')
    side = -1
    check_repeat = []
    FLIP = False
    done = False
    
    print(f'Episode = {episode}.')
    while True:
        observation = np.array(checkers)[np.newaxis,:]
        action_index = RL.choose_action(observation)
        action = RL.ACTION(action_index, np.array(checkers))
        
        if action[0]!=action[1]:
            #print(f'\tStep = {step}.')
            print(f'\tWhite King moves from {action[0]} to {action[1]}.\n\t====================================\n\tNot win yet...')
            checkers_next = RL.APPLY_ACTION(action, checkers)
            observation_ = np.array(checkers_next)[np.newaxis,:]
            if check_repeat!=[] and action[0]==check_repeat[0] and action[1]==check_repeat[1]:
                reward = -5.0
                print('REPEATING!!!!')
                done = True
            else:
                if checkers_next[55] == king_value or checkers_next[62] == king_value:
                    reward = 0.75
                side = -side
                check_repeat = [action[1], action[0]]

                checkers_next = COMPUTER_MOVE.REVERSE_COLOR_AND_BOARD(checkers_next)
                checkers_next, point = COMPUTER_MOVE.FIND_BEST_MOVE_ALPHA_BETA(-1, checkers_next, 4, alpha, beta) #alpha-beta
                checkers_next = COMPUTER_MOVE.REVERSE_COLOR_AND_BOARD(checkers_next)
                side = -side
                print(f'\tRed king at {np.argwhere(np.array(checkers_next)==-king_value)[0]} .')
                remain = len(np.argwhere(np.array(checkers_next)==king_value))
                if point == -max_eva:
                    reward = 10.0
                    done = True
                    total_win += 1
                    print('Win!')
                elif remain==1:
                    reward = -20.0
                    done = True
                    total_draw += 1
                    print('Draw!')
                elif remain==0:
                    reward = -20.0
                    done = True
                    total_lose += 1
                    print('Lose!')
                else:
                    reward = action[-1]
            if (step > 200) and (step % 5 == 0): ##大於兩百後每五步學習一次
                RL.learn()
            RL.store_transition(observation, action_index, reward, observation_)
            checkers = checkers_next.copy()
            if not done:
                print('\tNot win yet...')
        step += 1
        if done:
            break 
print(f'Winning rate is {total_win/(total_win+total_lose+total_draw)*100:.2f} %.')
print(f'Drawing rate is {total_draw/(total_win+total_lose+total_draw)*100:.2f} %.')
print(f'Losing rate is {total_lose/(total_win+total_lose+total_draw)*100:.2f} %.')

Episode = 0.
	White King moves from 12 to 19.
	Not win yet...
	Red king at [62] .
	Not win yet...
	White King moves from 19 to 12.
	Not win yet...
REPEATING!!!!
Episode = 1.
	White King moves from 26 to 35.
	Not win yet...
	Red king at [46] .
	Not win yet...
	White King moves from 35 to 42.
	Not win yet...
	Red king at [39] .
	Not win yet...
	White King moves from 42 to 33.
	Not win yet...
	Red king at [46] .
	Not win yet...
	White King moves from 12 to 19.
	Not win yet...
	Red king at [37] .
	Not win yet...
	White King moves from 33 to 26.
	Not win yet...
	Red king at [46] .
	Not win yet...
	White King moves from 26 to 35.
	Not win yet...
	Red king at [55] .
	Not win yet...
	White King moves from 19 to 12.
	Not win yet...
	Red king at [62] .
	Not win yet...
	White King moves from 35 to 42.
	Not win yet...
	Red king at [53] .
	Not win yet...
	White King moves from 12 to 19.
	Not win yet...
	Red king at [46] .
	Not win yet...
	White King moves from 19 to 10.
	Not win yet...
	Red king at

	White King moves from 7 to 14.
	Not win yet...
REPEATING!!!!
Episode = 10.
	White King moves from 3 to 10.
	Not win yet...
	Red king at [62] .
Epoch 1/10
128/128 [==============================] - 0s 156us/step - loss: 2.6439 - acc: 0.7812
Epoch 2/10
128/128 [==============================] - 0s 203us/step - loss: 2.1961 - acc: 0.8047
Epoch 3/10
128/128 [==============================] - 0s 171us/step - loss: 1.8276 - acc: 0.8203
Epoch 4/10
128/128 [==============================] - 0s 317us/step - loss: 1.5355 - acc: 0.7812
Epoch 5/10
128/128 [==============================] - 0s 148us/step - loss: 1.2500 - acc: 0.7656
Epoch 6/10
128/128 [==============================] - 0s 195us/step - loss: 1.0581 - acc: 0.8125
Epoch 7/10
128/128 [==============================] - 0s 284us/step - loss: 0.8903 - acc: 0.8516
Epoch 8/10
128/128 [==============================] - 0s 218us/step - loss: 0.7951 - acc: 0.8906
Epoch 9/10
128/128 [==============================] - 0s 179us/step - loss: 0.67

128/128 [==============================] - 0s 148us/step - loss: 0.1738 - acc: 0.8750
Epoch 7/10
128/128 [==============================] - 0s 125us/step - loss: 0.1393 - acc: 0.9219
Epoch 8/10
128/128 [==============================] - 0s 187us/step - loss: 0.1217 - acc: 0.9141
Epoch 9/10
128/128 [==============================] - 0s 156us/step - loss: 0.1041 - acc: 0.9141
Epoch 10/10
128/128 [==============================] - 0s 140us/step - loss: 0.0904 - acc: 0.9375
	Not win yet...
	White King moves from 40 to 33.
	Not win yet...
	Red king at [62] .
	Not win yet...
	White King moves from 28 to 19.
	Not win yet...
	Red king at [53] .
Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 0.2717 - acc: 0.937 - 0s 156us/step - loss: 0.2310 - acc: 0.9453
Epoch 2/10
128/128 [==============================] - 0s 140us/step - loss: 0.1886 - acc: 0.9375
Epoch 3/10
128/128 [==============================] - 0s 187us/step - loss: 0.1535 - acc: 0.9375
Epoch 4/10
128/128 [======

128/128 [==============================] - 0s 148us/step - loss: 0.5813 - acc: 0.8359
Epoch 9/10
128/128 [==============================] - 0s 140us/step - loss: 0.5059 - acc: 0.8281
Epoch 10/10
128/128 [==============================] - 0s 140us/step - loss: 0.4501 - acc: 0.8438
	Not win yet...
	White King moves from 51 to 58.
	Not win yet...
	Red king at [39] .
Epoch 1/10
128/128 [==============================] - 0s 171us/step - loss: 0.8608 - acc: 0.8906
Epoch 2/10
128/128 [==============================] - 0s 210us/step - loss: 0.7242 - acc: 0.8906
Epoch 3/10
128/128 [==============================] - 0s 179us/step - loss: 0.5794 - acc: 0.8516
Epoch 4/10
128/128 [==============================] - 0s 148us/step - loss: 0.4700 - acc: 0.8516
Epoch 5/10
128/128 [==============================] - 0s 195us/step - loss: 0.3918 - acc: 0.8750
Epoch 6/10
128/128 [==============================] - 0s 171us/step - loss: 0.3395 - acc: 0.8750
Epoch 7/10
128/128 [==============================] 

	Red king at [46] .
	Not win yet...
	White King moves from 49 to 40.
	Not win yet...
	Red king at [55] .
	Not win yet...
	White King moves from 40 to 33.
	Not win yet...
	Red king at [46] .
Epoch 1/10
128/128 [==============================] - 0s 117us/step - loss: 0.5999 - acc: 0.8281
Epoch 2/10
128/128 [==============================] - 0s 132us/step - loss: 0.5245 - acc: 0.7969
Epoch 3/10
128/128 [==============================] - 0s 203us/step - loss: 0.4594 - acc: 0.8125
Epoch 4/10
128/128 [==============================] - 0s 179us/step - loss: 0.3945 - acc: 0.8359
Epoch 5/10
128/128 [==============================] - 0s 195us/step - loss: 0.3551 - acc: 0.8516
Epoch 6/10
128/128 [==============================] - 0s 132us/step - loss: 0.3128 - acc: 0.8281
Epoch 7/10
128/128 [==============================] - 0s 148us/step - loss: 0.2839 - acc: 0.8438
Epoch 8/10
128/128 [==============================] - 0s 195us/step - loss: 0.2590 - acc: 0.8828
Epoch 9/10
128/128 [==============

...

	Red king at [37] .
	Not win yet...
	White King moves from 10 to 19.
	Not win yet...
	Red king at [30] .
	Not win yet...
	White King moves from 19 to 10.
	Not win yet...
REPEATING!!!!
Episode = 977.
	White King moves from 30 to 23.
	Not win yet...
	Red king at [62] .
Epoch 1/10
128/128 [==============================] - 0s 140us/step - loss: 1.5769 - acc: 0.7891
Epoch 2/10
128/128 [==============================] - 0s 171us/step - loss: 1.3693 - acc: 0.7891
Epoch 3/10
128/128 [==============================] - 0s 132us/step - loss: 1.1820 - acc: 0.7969
Epoch 4/10
128/128 [==============================] - 0s 125us/step - loss: 1.0179 - acc: 0.7969
Epoch 5/10
128/128 [==============================] - 0s 117us/step - loss: 0.8844 - acc: 0.8125
Epoch 6/10
128/128 [==============================] - 0s 117us/step - loss: 0.7801 - acc: 0.7969
Epoch 7/10
128/128 [==============================] - 0s 117us/step - loss: 0.6908 - acc: 0.8359
Epoch 8/10
128/128 [==============================] -

	Not win yet...
	Red king at [55] .
	Not win yet...
	White King moves from 14 to 5.
	Not win yet...
	Red king at [62] .
	Not win yet...
	White King moves from 39 to 30.
	Not win yet...
	Red king at [53] .
	Not win yet...
	White King moves from 30 to 39.
	Not win yet...
REPEATING!!!!
Episode = 983.
	White King moves from 3 to 10.
	Not win yet...
	Red king at [46] .
	Not win yet...
	White King moves from 10 to 19.
	Not win yet...
	Red king at [39] .
	Not win yet...
	White King moves from 19 to 12.
	Not win yet...
	Red king at [30] .
Epoch 1/10
128/128 [==============================] - 0s 101us/step - loss: 1.9171 - acc: 0.8281
Epoch 2/10
128/128 [==============================] - 0s 132us/step - loss: 1.7290 - acc: 0.8281
Epoch 3/10
128/128 [==============================] - 0s 117us/step - loss: 1.4735 - acc: 0.8281
Epoch 4/10
128/128 [==============================] - 0s 117us/step - loss: 1.2979 - acc: 0.8047
Epoch 5/10
128/128 [==============================] - 0s 117us/step - loss:

128/128 [==============================] - 0s 164us/step - loss: 1.4498 - acc: 0.8438
Epoch 2/10
128/128 [==============================] - 0s 140us/step - loss: 1.2433 - acc: 0.8281
Epoch 3/10
128/128 [==============================] - 0s 117us/step - loss: 1.0664 - acc: 0.8203
Epoch 4/10
128/128 [==============================] - 0s 132us/step - loss: 0.9064 - acc: 0.8281
Epoch 5/10
128/128 [==============================] - 0s 125us/step - loss: 0.8056 - acc: 0.8438
Epoch 6/10
128/128 [==============================] - 0s 117us/step - loss: 0.7235 - acc: 0.8047
Epoch 7/10
128/128 [==============================] - 0s 125us/step - loss: 0.6420 - acc: 0.7969
Epoch 8/10
128/128 [==============================] - 0s 117us/step - loss: 0.6056 - acc: 0.8125
Epoch 9/10
128/128 [==============================] - ETA: 0s - loss: 0.9735 - acc: 0.781 - 0s 125us/step - loss: 0.5535 - acc: 0.8203
Epoch 10/10
128/128 [==============================] - 0s 132us/step - loss: 0.5216 - acc: 0.8203
	N

128/128 [==============================] - 0s 234us/step - loss: 0.5678 - acc: 0.8750
Epoch 2/10
128/128 [==============================] - 0s 164us/step - loss: 0.4836 - acc: 0.8594
Epoch 3/10
128/128 [==============================] - 0s 125us/step - loss: 0.4040 - acc: 0.8984
Epoch 4/10
128/128 [==============================] - 0s 117us/step - loss: 0.3207 - acc: 0.9062
Epoch 5/10
128/128 [==============================] - 0s 117us/step - loss: 0.2756 - acc: 0.8906
Epoch 6/10
128/128 [==============================] - 0s 132us/step - loss: 0.2388 - acc: 0.8906
Epoch 7/10
128/128 [==============================] - 0s 140us/step - loss: 0.2099 - acc: 0.8828
Epoch 8/10
128/128 [==============================] - 0s 117us/step - loss: 0.1895 - acc: 0.8672
Epoch 9/10
128/128 [==============================] - 0s 148us/step - loss: 0.1684 - acc: 0.8672
Epoch 10/10
128/128 [==============================] - 0s 117us/step - loss: 0.1546 - acc: 0.8594
	Not win yet...
	White King moves from 1

	Not win yet...
	Red king at [53] .
	Not win yet...
	White King moves from 58 to 49.
	Not win yet...
	Red king at [62] .
Epoch 1/10
128/128 [==============================] - 0s 101us/step - loss: 1.8142 - acc: 0.8359
Epoch 2/10
128/128 [==============================] - 0s 218us/step - loss: 1.5722 - acc: 0.8438
Epoch 3/10
128/128 [==============================] - 0s 125us/step - loss: 1.2521 - acc: 0.8125
Epoch 4/10
128/128 [==============================] - 0s 132us/step - loss: 1.0934 - acc: 0.8359
Epoch 5/10
128/128 [==============================] - 0s 156us/step - loss: 0.8809 - acc: 0.8438
Epoch 6/10
128/128 [==============================] - 0s 187us/step - loss: 0.7655 - acc: 0.8438
Epoch 7/10
128/128 [==============================] - 0s 117us/step - loss: 0.6760 - acc: 0.8516
Epoch 8/10
128/128 [==============================] - 0s 132us/step - loss: 0.6066 - acc: 0.8594
Epoch 9/10
128/128 [==============================] - 0s 109us/step - loss: 0.5351 - acc: 0.8750
Epoch 

訓練的結果可以讓勝棋的概率提升到 18.07 %，被吃掉一子而被逼和的機率是 71.43 %，不過不幸的是輸棋的機率不為 0(如果用 $\alpha$-$\beta$ prouning 可能贏不了，但至少不會輸)，看起來是用這個訓練出來的殘局走棋方式是兩面刃呢~而且成效不是很好@@a


### Trained Model Weight Saving

In [31]:
checkers_model1_2nd_json = RL.model1.to_json()
open('./Checkers_RL_Model/RL_model1_2nd','w').write(checkers_model1_1st_json)
RL.model1.save_weights('./Checkers_RL_Model/RL_model1_2nd_weights.h5')

In [30]:
checkers_model2_2nd_json = RL.model2.to_json()
open('./Checkers_RL_Model/RL_model2_2nd','w').write(checkers_model2_1st_json)
RL.model2.save_weights('./Checkers_RL_Model/RL_model2_2nd_weights.h5')

### 最後我們把訓練好的 QLN 神經網路拿來和玩家對弈，觀察看看有什麼奇葩行為XD
這裡用訓練好的神經網路針對當前的殘局局面給出下一步，如前面所述的，動作是看輸出的 array 哪個元素最大且符合規則而做選擇。一旦神經網路一直重複某兩步(如 30-21 然後 21-30 )，就會陷入僵局，這時我們則改用 $\alpha$-$\beta$ prouning 為白棋做選擇。因為我們的神經網路沒有被訓練到吃棋，所以如果下一步白棋可以吃紅棋的話，就判斷白勝，如果紅棋可以吃白棋的話，則紅勝。

In [32]:
pg.init()

FLIP = False
FPS = 30 # frame per second
#clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append(0)
    
man_r = []
man_w = []
king_r, king_w = RL.POSITION_INITIALIZE()

#################
BOARD = Pygame_Plotting.Plotting(Nx, grid_size, margin_x, margin_y, Lx, Ly, man_value, king_value, FLIP, WINDOWS)
CHECKERS = Basic_Operation.Checkers_Basic(Nx, man_value, king_value, step_fr_r, step_fl_r, step_br_r, step_bl_r, \
                                          step_fr_w, step_fl_w, step_br_w, step_bl_w, man_r, man_w, king_r, king_w)
COMPUTER_MOVE = Mini_Max_and_Alpha_Beta.prouning(Nx, max_eva, man_value, king_value, CHECKERS)

checkers = CHECKERS.CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#RL.model1.load_weights('./Checkers_RL_Model/RL_model1_1st_weights.h5')
RL.model1.load_weights('./Checkers_RL_Model/RL_model1_2nd_weights.h5')
    
BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
pg.display.flip()

click= []
save = []
check_repeat = []

side = -1
total_move = 1
depth = 6
if side == 1:
    red_first = True
else:
    red_first = False

point = 0.0
running = True
continue_to_jump = False

if side==1:
    print('Red Turn')
    CHECKERS.saving(checkers, save)
else:
    print('White Trun')
print('==============================================')

while running:
    event = pg.event.wait()
    mods = pg.key.get_mods()
    if event.type == MOUSEBUTTONDOWN and event.button==1:
        (x,y) = pg.mouse.get_pos()
        column = (x-margin_x)//grid_size
        row = (y-margin_y)//grid_size
        index = column + row*Nx
        if FLIP:
            index = Nx**2-1-index
        click.append(index)
    # take back
    #elif not continue_to_jump and event.type == KEYDOWN and mods&pg.KMOD_CTRL:  # See https://www.pygame.org/docs/ref/key.html
    elif event.type == KEYDOWN and mods&pg.KMOD_CTRL:    
        if event.key == pg.K_z:
            if len(save)>1:
                checkers = save[-2].copy()  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                            # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html
                del save[-1]  # remove the last element
        
                BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
                BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip() 
                print('\tRed takes back.\n==============================================\nRed Turn')
                total_move -= 1
        elif event.key == pg.K_f:
            FLIP = not FLIP
            print('Board flipped.')
            BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
            BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
            pg.display.flip() 
     
    # close the window
    elif event.type == pg.QUIT:
        running = False
        
    #if not continue_to_jump:
    man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king = CHECKERS.AVAILABLE_MOVE(side, checkers)
    available_move = man_jump+king_jump+man_walk+king_walk
    # draw out the hints
    (available_start,available_stop) = BOARD.MOVE_HINT(available_move)  # find the avaiable start points and stop points, then drawing it out as hint
    if side==1:
        for i in available_start:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,0,255),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        for i in available_stop:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,255,0),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        pg.display.flip()
    # have a winner
    if king_jump != []:
        if side==-1:
            white_win = True
            print('White can jump.')
            print('White win!')
        else:
            red_win = True
            print('Red can jump.')
            print('Red win!')
        break

    # Computer moves
    if side==-1:
        print('\tComputer moves...')
        
        action_array = RL.model1.predict(np.array(checkers)[np.newaxis,:]).reshape(-1)
        action_array -= action_array.min()
        action_index = np.argmax(action_array)
        action = RL.ACTION(action_index, np.array(checkers))
        while action[0]==action[1]:
            action_array[action_index] = -1
            action_index = np.argmax(action_array)
            action = RL.ACTION(action_index, np.array(checkers))
        if check_repeat!=[] and action[0]==check_repeat[0] and action[1]==check_repeat[1]:
            print('\tRepeating! Changes to alpha-beta prouning!')
            checkers, point = COMPUTER_MOVE.FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta) #alpha-beta
        else:
            checkers = RL.APPLY_ACTION(action, checkers)
            check_repeat = [action[1], action[0]]
        
        print('\tComputer finishes move.')
        if red_first:
            print(f'Move {total_move} Ends')
            total_move += 1
        print('Red Turn\n==============================================')
        
        side = -side
        BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
        BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP) 
        pg.display.flip()
        CHECKERS.saving(checkers,save)

    # read the last two clicked square and see whether it corresponding any available move
    elif side==1 and len(click)>1:
        # king walk
        if tuple(click[-2:]) in king_walk:
            start = click[-2]
            stop = click[-1]              
            checkers[start] = 0
            checkers[stop] = -king_value
            print('\tKing walks from %d to %d.' %(start,stop))
            side = -side
            if not red_first:
                print(f'Move {total_move} Ends')
                total_move += 1
            print('White Trun\n==============================================')
            click= []
            BOARD.WINDOWS_PLOT(WINDOWS, FLIP)
            BOARD.CHECKER_PLOT(checkers, WINDOWS, FLIP)
            pg.display.flip()
pg.quit()

White Trun
	Computer moves...
	Computer finishes move.
Red Turn
	King walks from 55 to 62.
Move 1 Ends
White Trun
	Computer moves...
	Computer finishes move.
Red Turn
	King walks from 62 to 55.
Move 2 Ends
White Trun
	Computer moves...
	Computer finishes move.
Red Turn
	King walks from 55 to 62.
Move 3 Ends
White Trun
	Computer moves...
	Computer finishes move.
Red Turn
Red can jump.
Red win!


測試結果感覺是白棋會主動接近並壓迫紅方的棋子，如果紅方主動走到邊邊的話就會主動上來把紅棋關廁所，但也會時不時的送子造成和局甚至輸棋，感覺送子之後給的負 reward 沒發揮什麼效果，也沒有防守的概念，看來用 QLN 來收殘局好像效果不是頂好的(或是我們的訓練方式或 reward 沒設計好 QQ)...